In [1]:
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle  # Add this line

from sae_cooccurrence.utils.set_paths import get_git_root

In [2]:
# Define the indices of interest
# indices = [9869, 8129, 6449, 13989, 1469, 13623, 10032, 7407, 9590, 6854]

model_name = "gemma-2-2b"
sae_release = "gemma-scope-2b-pt-res-canonical"
sae_id = "layer_0/width_16k/canonical"
sae_id_safe = sae_id.replace("/", "_").replace(".", "_")
layer_idx = 0
n_examples = 10000

# Define indices to highlight
highlight_indices = [1469, 6449, 8129, 13989]

top_sae_indices_path = os.path.join(
    get_git_root(),
    "results",
    "linear_probes",
    model_name,
    sae_release,
    sae_id,
    f"top_sae_indices_n_examples_{n_examples}.txt",
)
feature_indices_df = pd.read_csv(
    top_sae_indices_path
)  # Adjust the path and file type as necessary
top_sae_indices = feature_indices_df[
    "feature_index_column"
].tolist()  # Replace 'feature_index_column' with the actual column name


# Get positions of highlighted indices in our matrix
highlight_positions = [top_sae_indices.index(idx) for idx in highlight_indices]

# Load the NPZ file
data = np.load(
    "../results/gemma-2-2b/gemma-scope-2b-pt-res-canonical/layer_0_width_16k_canonical/n_batches_100/feature_acts_cooc_total_threshold_1_5.npz"
)
matrix = data["arr_0"]

# Extract the submatrix
submatrix = matrix[top_sae_indices][:, top_sae_indices]

# set diagonal to zero and put upper triangular
np.fill_diagonal(submatrix, 0)
submatrix = np.triu(submatrix)

# Create the plot
plt.figure(figsize=(10, 8))
ax = sns.heatmap(
    submatrix,
    cmap="viridis",
    xticklabels=[str(idx) for idx in top_sae_indices],
    yticklabels=[str(idx) for idx in top_sae_indices],
    annot=True,
    fmt=".2f",
    square=True,
)

# Add rectangles around highlighted cells
for i in highlight_positions:
    for j in highlight_positions:
        ax.add_patch(Rectangle((j, i), 1, 1, fill=False, edgecolor="red", lw=2))

plt.title(
    "Feature Co-occurrence Matrix for Selected Features\nRed boxes highlight selected features"
)
plt.tight_layout()
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: '../results/gemma-2-2b/gemma-scope-2b-pt-res-canonical/layer_0_width_16k_canonical/n_batches_100/feature_acts_cooc_total_threshold_1_5.npz'